In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cve-dataset-with-embeddings-using-bert/CVE_embeddings_bert.csv


In [2]:
df = pd.read_csv('/kaggle/input/cve-dataset-with-embeddings-using-bert/CVE_embeddings_bert.csv')


/tmp/ipykernel_17/964884605.py:1: DtypeWarning: Columns (40,41,44,45,47,48,52,53,54,55,56,57,58,59,60,62,63,64,65,69,70,71,72,73,74,75,76,77,79,80,81,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/cve-dataset-with-embeddings-using-bert/CVE_embeddings_bert.csv')


In [3]:
df.columns[:87]


Index(['cve.id', 'cve.sourceIdentifier', 'cve.published', 'cve.lastModified',
       'cve.vulnStatus', 'cve.weaknesses', 'cve.metrics.cvssMetricV30',
       'cve.metrics.cvssMetricV31', 'cve.weaknesses.primary.source',
       'cve.weaknesses.primary.description', 'cve.weaknesses.secondary.source',
       'cve.weaknesses.secondary.description', 'cve.descriptions.lang.en',
       'cve.descriptions.lang.es', 'cve.metrics.cvssMetricV2.source',
       'cve.metrics.cvssMetricV2.type',
       'cve.metrics.cvssMetricV2.baseSeverity',
       'cve.metrics.cvssMetricV2.exploitabilityScore',
       'cve.metrics.cvssMetricV2.impactScore',
       'cve.metrics.cvssMetricV2.acInsufInfo',
       'cve.metrics.cvssMetricV2.obtainAllPrivilege',
       'cve.metrics.cvssMetricV2.obtainUserPrivilege',
       'cve.metrics.cvssMetricV2.obtainOtherPrivilege',
       'cve.metrics.cvssMetricV2.userInteractionRequired',
       'cve.metrics.cvssMetricV2.cvssData.version',
       'cve.metrics.cvssMetricV2.cvssData.v

In [4]:
dfi = pd.concat([df.iloc[:, 87:], df[['cve.metrics.cvssMetricV2.obtainOtherPrivilege']]], axis=1)

In [5]:
dfi.head()

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767,cve.metrics.cvssMetricV2.obtainOtherPrivilege
0,-0.426594,-0.664082,-0.311982,-0.201928,-0.199103,-0.067311,0.686270,0.389902,-0.356045,-0.285820,...,-0.054814,0.115219,0.063408,0.357258,0.397121,-0.171131,-0.041268,-0.373622,0.189914,False
1,-0.346958,-0.684867,-0.188105,-0.115072,-0.135127,-0.753387,0.300061,0.347360,-0.253761,0.125624,...,0.066620,0.232606,-0.141562,-0.140026,0.582920,0.013143,-0.514797,-0.488033,0.732211,False
2,-0.543853,-0.726613,0.020474,-0.276225,-0.286479,-0.231333,0.015804,0.195252,-0.520817,-0.264572,...,-0.011729,0.081952,0.054252,0.077415,0.097131,-0.123902,-0.599761,-0.099518,0.403333,False
3,-0.511614,-0.493360,0.172049,-0.267262,-0.007668,0.066099,0.524465,0.301235,-0.227156,-0.572937,...,-0.066118,0.212358,-0.008162,0.202250,0.061845,-0.253034,-0.268757,-0.198453,0.415803,False
4,-0.474302,-0.114922,-0.167968,-0.281234,-0.198231,-0.267894,0.611416,0.363515,-0.105000,-0.513623,...,-0.145129,0.294382,-0.000997,-0.118320,0.369955,0.003200,-0.552267,-0.346569,0.775922,False


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
import pickle

# Assuming dfi is your DataFrame
X = dfi.iloc[:, :-1]  # Features
y = dfi.iloc[:, -1]   # Target labels

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert object columns to numeric using LabelEncoder or one-hot encoding
for column in X.columns:
    if X[column].dtype == 'object':
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}")
    print(f"{model_name} R² Score: {r2:.4f}\n")

    # Save the model as a pickle file
    with open(f'{model_name.lower()}_model.pkl', 'wb') as f:
        pickle.dump(model, f)

# Initialize models
models = {
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the label encoder as well, in case you need to decode predictions later
with open('label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)


XGBoost Accuracy: 0.8085
XGBoost Mean Squared Error: 0.1915
XGBoost Mean Absolute Error: 0.1915
XGBoost R² Score: -0.2176

Random Forest Accuracy: 0.8145
Random Forest Mean Squared Error: 0.1855
Random Forest Mean Absolute Error: 0.1855
Random Forest R² Score: -0.1794

AdaBoost Accuracy: 0.7915
AdaBoost Mean Squared Error: 0.2085
AdaBoost Mean Absolute Error: 0.2085
AdaBoost R² Score: -0.3257



In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import pickle

# Assuming dfi is your DataFrame
X = dfi.iloc[:, :-1]  # Features
y = dfi.iloc[:, -1]   # Target labels

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}\n")

    # Save the model as a pickle file
    with open(f'{model_name.lower()}_model.pkl', 'wb') as f:
        pickle.dump(model, f)

# Initialize models
models = {
    'Support Vector Machine': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the label encoder as well, in case you need to decode predictions later
# with open('label_encoder.pkl', 'wb') as le_file:
#     pickle.dump(label_encoder, le_file)


Support Vector Machine Accuracy: 0.8045
Support Vector Machine Mean Squared Error: 0.1955
Support Vector Machine Mean Absolute Error: 0.1955

Gradient Boosting Accuracy: 0.8125
Gradient Boosting Mean Squared Error: 0.1875
Gradient Boosting Mean Absolute Error: 0.1875

